In [1]:
!nvidia-smi

Tue Apr 26 15:57:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import h5py
import numpy as np
from tqdm.auto import tqdm
import random
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [3]:
batch_size = 100
mini_batch_size = 50
lr_G = 0.0025
lr_D = 0.0001
beta1 = 0.5
workers = 0
dataset_name = 'shapenet_v2'
obj = 'airplane'
dim = 128
noise_dim = 200 # latent space vector dim
in_channels = 512 # convolutional channels
run_parallel = False

Random Seed:  42
batch size: 100 mini batch: 50 k: 2
dim: 128
device: cuda


In [ ]:
k = int(batch_size / mini_batch_size)
print('batch size:', batch_size, 'mini batch:', mini_batch_size, 'k:', k)

# Set random seed for reproducibility
manualSeed = 42
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

inverse_scale = 1
print('dim:', dim)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device:', device)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
project_path = '/content/drive/MyDrive/diss/3D-GAN'
data_path = f'{project_path}/data'
data_filename = f'{dataset_name}_{obj}_r{dim}'
weights_path = f'{project_path}/weights/{data_filename}'
src_path = f'{project_path}/3D-GAN'

os.makedirs(weights_path, exist_ok=True)

In [ ]:
f = h5py.File(f'{data_path}/{data_filename}.h5', 'r')
dataset = torch.from_numpy(np.array(f[list(f.keys())[0]]).reshape(-1, 1, dim, dim, dim)).to(torch.float)
if inverse_scale > 1:
    assert type(inverse_scale)==int
    dataset = dataset[:, :, ::inverse_scale, ::inverse_scale, ::inverse_scale]

print('dataset shape:', dataset.shape)

In [ ]:
dataloader = DataLoader(
    dataset, 
    batch_size=batch_size,
    shuffle=True, 
    num_workers=workers,
)

print('number of batches:', len(dataloader))

# GAN Structure 

In [ ]:
import sys
sys.path.append(src_path) # adding path to sys

from src.GAN import Discriminator, Generator, weights_init

netG = Generator(in_channels=256, out_dim=dim, out_channels=1, noise_dim=noise_dim)
if run_parallel:
    netG = torch.nn.DataParallel(netG)
netG = netG.to(device)
netG.apply(weights_init)
# noise = torch.rand(1, noise_dim).to(device)
# generated_volume = netG(noise)
# print("Generator output shape", generated_volume.shape)
netD = Discriminator(in_channels=1, out_conv_channels=256, dim=dim)
if run_parallel:
    netD = torch.nn.DataParallel(netD)
netD = netD.to(device)
netD.apply(weights_init)

criterion = torch.nn.BCELoss()
# # Establish convention for real and fake labels during training
# real_label = 1.
# fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr_D, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr_G, betas=(beta1, 0.999))

# out = netD(generated_volume)
# print("Discriminator output", out.item())

print("\n\nGenerator summary\n\n")
summary(netG, (1, noise_dim))
print("\n\nDiscriminator summary\n\n")
summary(netD, (1, dim, dim, dim))

# Training and Testing 3D-GAN

# Running Epochs

In [ ]:
def plot_convergence(G_losses, D_real_losses, D_fake_losses, real_accuracies, fake_accuracies):
  lst_epoch = np.array(range(iters* len(data_split))) / len(dataloader)
  plt.figure(figsize=(10,5))
  plt.title("Generator and Discriminator Loss During Training")
  plt.plot(lst_epoch, G_losses,label="G")
  plt.plot(lst_epoch, D_real_losses, label="D_real")
  plt.plot(lst_epoch, D_fake_losses, label="D_fake")
  plt.xlabel("Epoch")
  plt.ylabel("Loss")
  plt.legend()
  plt.show()

  plt.figure(figsize=(10,5))
  plt.title("Discriminator Accuracies During Training")
  plt.plot(lst_epoch, real_accuracies, label="acc_real")
  plt.plot(lst_epoch, fake_accuracies, label="acc_fake")
  plt.xlabel("Epoch")
  plt.ylabel("Accuracy")
  plt.legend()
  plt.show()

In [ ]:
# Lists to keep track of progress
G_losses = []
D_real_losses = []
D_fake_losses = []
real_accuracies = []
fake_accuracies = []
start_epoch = 0
iters = 0

real_label = 1.
fake_label = 0.

os.makedirs(f'{project_path}/weights/{data_filename}', exist_ok=True)

In [ ]:
# successful_weights_path = f'{project_path}/weights/successful/{data_filename}'
# start_epoch = 490
# file_netG = f'{successful_weights_path}/netG_e{start_epoch}_r{dim}_weights.pth'
# print(file_netG)
# netG.load_state_dict(torch.load(file_netG))
# file_netD = f'{successful_weights_path}/netD_e{start_epoch}_r{dim}_weights.pth'
# print(file_netD)
# netD.load_state_dict(torch.load(file_netD))

# print('netG and netD weights set to previous successful ones')

In [ ]:
num_epochs = 100

# Training Loop
print("Starting Training Loop...")
# For each epoch
for epoch in tqdm(range(start_epoch, start_epoch+num_epochs)):
    # For each batch in the dataloader
    lst_train_acc_real = []
    lst_train_acc_fake = []
    for i, data_all in enumerate(dataloader, 0):
        data_split = torch.split(data_all, mini_batch_size)
        optimizerD.zero_grad()
#         print('reset netD grads')
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        # Format batch
        for j in range(len(data_split)):
            data = data_split[j]
#             print(data.shape)
            real_cpu = data.to(device)
            b_size = real_cpu.size(0)
            label_real = torch.full((b_size,), real_label, dtype=torch.float, device=device)
            label_fake = torch.full((b_size,), fake_label, dtype=torch.float, device=device)
            # Forward pass real batch through D
            output_real = netD(real_cpu).view(-1)
            # Calculate loss on all-real batch
            D_x = output_real.mean().item()
            train_acc_real = torch.sum((output_real > 0.5).to(int)==label_real) / b_size
            lst_train_acc_real.append(train_acc_real.item())
            errD_real = criterion(output_real, label_real) / len(data_split)
            errD_real.backward()

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, noise_dim, device=device)
            # Generate fake image batch with G
            fake = netG(noise).detach()
            output_fake = netD(fake).view(-1)
            D_G_z1 = output_fake.mean().item()
            train_acc_fake = torch.sum((output_fake > 0.5).to(int) == label_fake) / b_size
            lst_train_acc_fake.append(train_acc_fake.item())
            errD_fake = criterion(output_fake, label_fake) / len(data_split)
            errD_fake.backward()

            errD = errD_real + errD_fake        
        
        # update D only if classification acc is less than 80% for stability
#         if (i+1) % k == 0 or (i+1) == len(dataloader):
            if j==len(data_split)-1:
                acc_real_mean = np.mean(lst_train_acc_real)
                acc_fake_mean = np.mean(lst_train_acc_fake)
                update = ((acc_real_mean + acc_fake_mean) / 2) < 0.8
                if update:
                    optimizerD.step()  # update the weights only after accumulating k small batches
#                     print('updated optD')

                optimizerD.zero_grad()  # reset gradients for accumulation for the next large batch
                lst_train_acc_real = []
                lst_train_acc_fake = []
            
        

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        # fake labels are real for generator cost
        optimizerG.zero_grad()
#         print('reset netG grads')
        for j in range(len(data_split)):
            label = torch.full((b_size,), real_label, dtype=torch.float, device=device)  
            # Since we just updated D, perform another forward pass of all-fake batch through D
            fake = netG(noise)
            output = netD(fake).view(-1)
            errG = criterion(output, label) / len(data_split)
            errG.backward()


            D_G_z2 = output.mean().item()
#             if (i+1) % k == 0 or (i+1) == len(dataloader):
            if j==len(data_split)-1:
                optimizerG.step()  # update the weights only after accumulating k small batches
                optimizerG.zero_grad()  # reset gradients for accumulation for the next large_batch
#                 print('updated optG')

            # Save Losses for plotting later
            G_losses.append(errG.item())
            D_fake_losses.append(errD_fake.item())
            D_real_losses.append(errD_real.item())
            fake_accuracies.append(train_acc_fake.item())
            real_accuracies.append(train_acc_real.item())

        # Output training stats
        if i % 10 == 0: # print progress every epoch
            print(f'[{epoch}/{start_epoch+num_epochs}][{i}/{len(dataloader)}]\tLoss_D: {round(errD.item(), 4)}\tLoss_G: {round(errG.item(), 4)}\tD(x): {round(D_x, 4)}\tD(G(z)): {round(D_G_z1, 4)} / {round(D_G_z2, 4)}')
            
        iters += 1
    
    if epoch % 10 == 0 and epoch != 0:
      plot_convergence(G_losses, D_real_losses, D_fake_losses, real_accuracies, fake_accuracies)
      # save network weights
      netG_filename = f'{weights_path}/netG_e{epoch}_r{dim}_weights.pth'
      netD_filename = f'{weights_path}/netD_e{epoch}_r{dim}_weights.pth'
      torch.save(netG.state_dict(), netG_filename)
      torch.save(netD.state_dict(), netD_filename)
      print('saved network weights', netG_filename)


torch.save(netG.state_dict(), f'{weights_path}/netG_e{epoch}_r{dim}_weights.pth')
torch.save(netD.state_dict(), f'{weights_path}/netD_e{epoch}_r{dim}_weights.pth')
start_epoch = epoch # change start to the current

# Results 

## Convergence Graph

In [ ]:
plot_convergence(G_losses, D_real_losses, D_fake_losses, real_accuracies, fake_accuracies)

# Visualisation

## Real Samples

In [ ]:
real_sample = next(iter(dataloader))

In [ ]:
for i in range(5):
    s = real_sample[i][0]
    ax = plt.figure().add_subplot(projection='3d')
#     ax = plt.figure(figsize=(10, 10)).add_subplot(projection='3d')
    ax.voxels(s)
    plt.show()

# Generate fake samples

In [ ]:
fake_samples = []
n_fake_samples = 5

for i in tqdm(range(10, epoch, 10)):    
    try:
        file_netG = f'{weights_path}/netG_e{i}_r{dim}_weights.pth'
        print(file_netG)
        netG.load_state_dict(torch.load(file_netG))

        fixed_noise = torch.randn(n_fake_samples, noise_dim, device=device)
        with torch.no_grad():
            fake = netG(fixed_noise).detach().cpu().numpy()
        fake_samples.append(fake)
        # print('generated fake samples')
        
        
            
    except Exception as e:
        print(e)
        print('epoch', i, 'failed')

fake_samples = np.array(fake_samples)
fake_samples = fake_samples >= 0.5
print('fake sample shape:', fake_samples.shape)

In [ ]:
fake_samples_path = f'{data_path}/fake_samples'
os.makedirs(fake_samples_path, exist_ok=True)
h5_filename = f'{fake_samples_path}/fake_samples_{data_filename}.h5'
with h5py.File(h5_filename, "w") as f:
    dset = f.create_dataset("tensor", data=fake_samples)
    print(h5_filename, 'saved')

In [ ]:
for i in range(0, fake_samples.shape[0]):
  print('epoch', (i+1)*10)
  for j in range(fake_samples.shape[1]):
    voxel_grid = fake_samples[i][j][0] > 0.5
    ax = plt.figure().add_subplot(projection='3d')
    ax.voxels(voxel_grid)
    plt.show()
